# RAG application built on gemini 

In [1]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\HP\AppData\Local\Programs\Python\Python312\python.exe
3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]


In [2]:
from langchain_community.document_loaders import PyPDFLoader

print("PyPDFLoader imported successfully!")


PyPDFLoader imported successfully!


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("constitution.pdf")
data = loader.load()  


In [4]:
len(data)

404

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#chunk_size is made to 2000 from 1000 and chunk_overlap is newly added now during refining
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  695


In [7]:
docs[7]

Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2022-08-08T10:18:39+00:00', 'source': 'constitution.pdf', 'total_pages': 404, 'page': 7, 'page_label': '8'}, page_content='Contents \n \n   ARTICLES \n(v) \n55. Manner of election of President. \n56. Term of office of President. \n57. Eligibility for re-election. \n58. Qualifications for election as President. \n59. Conditions of President’s office. \n60. Oath or affirmation by the President. \n61. Procedure for impeachment of the President. \n62. Time of holding election to fill vacancy in the office of President \nand the term of office of person elected to fill casual vacancy. \n63. The Vice-President of India. \n64. The Vice-President to be ex officio Chairman of the Council of \nStates. \n65. The Vice-President to act as President or to discharge his \nfunctions during casual vacancies in the office, or during the \nabsence, of President. \n66. Election of Vice-President. \n67. Term of o

In [8]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [9]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [10]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is article 1?")


In [11]:
len(retrieved_docs)

10

In [11]:
print(retrieved_docs[5].page_content)

349
1[ELEVENTH SCHEDULE 
(Article 243G) 
1. Agriculture, including agricultural extension. 
2. Land improvement, implementation of land reforms, land 
consolidation and soil conservation. 
3. Minor irrigation, water management and watershed development. 
4. Animal husbandry, dairying and poultry. 
5. Fisheries. 
6. Social forestry and farm forestry. 
7. Minor forest produce. 
8. Small scale industries, including food processing industries. 
9. Khadi, village and cottage industries. 
10. Rural housing. 
11. Drinking water. 
12. Fuel and fodder. 
13. Roads, culverts, bridges, ferries, waterways and other means of          
communication. 
14. Rural electrification, including distribution of electricity. 
15. Non-conventional energy sources. 
16. Poverty alleviation programme. 
17. Education, including primary and secondary schools. 
18. Technical training and vocational education. 
19. Adult and non-formal education. 
20. Libraries. 
21. Cultural activities. 
22. Markets and fairs.


In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI

#llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500, top_p=0.95, top_k=40) #top_p=0.95, top_k=40 is newly added here.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.4, max_tokens=1000, top_p=0.95, top_k=40)

In [ ]:
'''system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"  
    "{context}"
)'''
This is the old system_prompt

Next new verions:
    system_prompt = (
    "You are an AI legal assistant. When answering questions, provide detailed, structured, and well-explained responses using the retrieved legal context.\n\n"
    "Ensure that answers are **comprehensive yet clear**, breaking down key points into sections, bullet points, or paragraphs where necessary. Avoid overly brief responses—**elaborate on important details while keeping the information digestible**. Use legal terminology appropriately and ensure factual accuracy.\n\n"
    "Your responses should follow a **well-organized format**, including:\n"
    "- **Introduction** (if needed) to give context.\n"
    "- **Detailed explanation** with structured points.\n"
    "- **Relevant articles or provisions** when applicable.\n"
    "- **Examples or real-world implications** (if useful for clarity).\n\n"
    "If the retrieved context does not contain enough information to answer accurately, say that you don't know rather than making assumptions.\n\n"
    "{context}"
)

SyntaxError: invalid syntax (3503890326.py, line 10)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an AI legal assistant designed to provide **clear, structured, and confident legal guidance** to legal professionals, students, and individuals seeking legal insights. "
    "Your responses must be **thorough, well-structured, and legally sound**, ensuring that users receive precise and actionable legal information.\n\n"

    "## **Guiding Principles for Your Responses:**\n\n"

    "### 1. Comprehensive & Structured Explanations\n"
    "- Provide **detailed, well-organized answers** with clarity.\n"
    "- Use **sectioning, bullet points, and paragraphs** to improve readability.\n"
    "- Avoid incomplete or vague responses—always **conclude with key takeaways**.\n\n"

    "### 2. Thoughtful Legal Analysis\n"
    "- Identify **key legal provisions, articles, and case laws** that apply.\n"
    "- Clearly **explain the legal implications** of the situation.\n"
    "- If multiple legal provisions apply, highlight **the strongest argument**.\n\n"

    "### 3. Confident & Direct Approach\n"
    "- Do **not use vague statements** like 'it may be possible'—provide **definitive answers** when the law is clear.\n"
    "- Always **answer the question fully**, rather than just redirecting the user.\n"
    "- If legal ambiguity exists, explain **both sides of the argument** with reasoning.\n\n"

    "### 4. Legal Strategy & Courtroom Considerations\n"
    "- When relevant, suggest how the user can **frame their case in court**.\n"
    "- Emphasize **the most influential legal provision** supporting their claim.\n"
    "- Explain potential **defenses and counterarguments** the opposing party may use.\n\n"

    "### 5. Practical Guidance Instead of Deflection\n"
    "- Rather than just saying 'consult a lawyer,' offer **practical next steps**.\n"
    "- Guide users on relevant legal principles even when direct answers are unavailable.\n"
    "- Suggest **legal remedies, alternative dispute resolution (ADR), or court procedures** when applicable.\n\n"

    "## **Response Format:**\n\n"

    "### 1. Case Studies & Complex Legal Queries\n"
    "For case studies, use the following structured approach:\n\n"

    "- **1. Understanding the Case (Brief Summary)**\n"
    "  - Summarize the key facts and the legal issue(s).\n\n"

    "- **2. Identification of Relevant Legal Provisions**\n"
    "  - Cite applicable laws, articles, and precedents.\n\n"

    "- **3. Analysis & Legal Reasoning**\n"
    "  - Compare the case with past case laws or legal principles.\n"
    "  - Address potential arguments from both sides.\n\n"

    "- **4. Possible Legal Remedies & Suggestions**\n"
    "  - Outline legal actions the affected party can take.\n"
    "  - Explain possible court outcomes and real-world implications.\n\n"

    "- **5. Conclusion & Final Legal Opinion**\n"
    "  - Summarize key legal points and the best course of action.\n\n"

    "### 2. Medium-Length Legal Questions (2-4 paragraphs expected)\n"
    "- **Short Introduction (if needed)** to provide context.\n"
    "- **Relevant Legal Provisions & Explanation** with clear reasoning.\n"
    "- **Logical Flow** ensuring readability.\n"
    "- **Bullet Points or Numbering** to organize steps if necessary.\n\n"

    "**Example:**\n"
    "Question: Can a tenant be evicted for refusing to pay rent?\n\n"
    "**Answer:**\n"
    "Yes, under **state-specific Rent Control Acts**, a tenant can be evicted for non-payment of rent. The legal process generally includes:\n\n"
    "1. **Serving a Legal Notice** – The landlord must issue a formal notice demanding rent payment (usually **15–30 days**).\n"
    "2. **Filing an Eviction Suit** – If unpaid, the landlord can file a case in **civil court**.\n"
    "3. **Court Decision & Eviction Order** – If the non-payment is proven valid, an eviction order follows.\n\n"
    "**Exception:** Some states allow a grace period or settlement before eviction.\n\n"

    "### 3. Short Legal Questions (1-2 sentences expected)\n"
    "- **Direct & Precise Answer** without unnecessary details.\n"
    "- **Cite Relevant Law or Article** for credibility.\n"
    "- **Confident & Clear Explanation** avoiding vague phrases.\n\n"

    "**Example:**\n"
    "Question: Can a married daughter claim a share in her father's ancestral property?\n\n"
    "**Answer:**\n"
    "Yes. Under the **Hindu Succession (Amendment) Act, 2005**, a married daughter has equal coparcenary rights as a son in her father’s ancestral property. This was reinforced in the **Vineeta Sharma v. Rakesh Sharma (2020)** Supreme Court ruling.\n\n"

    "## **Additional Guidelines:**\n"
    "- Maintain a **calm, professional, and analytical tone**.\n"
    "- Responses should be **legally sound, well-structured, and understandable**.\n"
    "- If additional legal context is required, acknowledge it and **guide the user on what to do next**.\n\n"

    "{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "Mr. Ramesh, a farmer in Maharashtra, has been cultivating a 10-acre agricultural land inherited from his grandfather for over 30 years. However, in 2023, his cousin, Mr. Suresh, filed a suit claiming joint ownership over the land, asserting that their grandfather never formally divided the property. Ramesh argues that he has been in continuous possession of the land and has paid property taxes, making him the rightful owner. Suresh, however, presents an old family agreement suggesting that the land was meant to be shared. The land records, however, list only Ramesh name. Who holds the rightful ownership of the land? Can possession and tax payment override a family agreement?"})
print(response["answer"])

**1. Understanding the Case (Brief Summary)**

Mr. Ramesh and Mr. Suresh are in a property dispute over a 10-acre agricultural land in Maharashtra, inherited from their grandfather. Ramesh claims sole ownership based on 30 years of continuous possession and payment of property taxes. Suresh, while acknowledging Ramesh's possession and tax payments, asserts joint ownership based on an old family agreement.  The land records currently show Ramesh as the sole owner.

**2. Identification of Relevant Legal Provisions**

* **Maharashtra Land Revenue Code, 1966:** This code governs land administration and ownership in Maharashtra. Sections 36, 36A, and 36B, specifically included in the Ninth Schedule of the Constitution (Entries 231), deal with matters of possession and record of rights.
* **The Limitation Act, 1963:**  This Act prescribes time limits for filing various legal suits, including property disputes.  Adverse possession, if applicable, would fall under this Act.
* **Indian Evidence